# Neo Bank Data Preparation Notebook

The purpose of this notebook is to clean the dataset and create the features used in the train/test datasets

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from datetime import timedelta

### Load historical (2008-2023) dataset used to train the model

In [2]:
import pandas as pd

dataframes = []

for year in range(2008, 2024):
    file_path = f"../data/train_{year}.parquet"
    try:
        df = pd.read_parquet(file_path)
        dataframes.append(df)
    except Exception as e:
        print(f"Failed to read file {file_path}: {e}")
if dataframes:
    combined_train_raw_df = pd.concat(dataframes, ignore_index=True)
    print(f"Total rows in the combined DataFrame: {len(combined_train_raw_df)}")
else:
    print("No files were successfully read.")

Total rows in the combined DataFrame: 3926058


### Load test dataset (2024)

In [3]:
file_path = f"../data/test.parquet"
test_raw_df = pd.read_parquet(file_path)
print(len(test_raw_df))

1360472


### Data Cleaning

In [4]:
# # drop unrealistic records of atm transfer out

# # current justification: it's not realisitc that someone can transfer money out of an ATM thousands of times in single day also it accounts for tiny 
# # TODO: find a source to backup the reasoning for removing this data

# initial_count = len(combined_train_raw_df)

# # drop records with more than 10 atm transfer outs
# filtered_df = combined_train_raw_df.drop(combined_train_raw_df[combined_train_raw_df['atm_transfer_out'] > 20].index)

# # Calculate dropped records
# dropped_count = initial_count - len(filtered_df)
# percentage_dropped = (dropped_count / initial_count) * 100

# print(f"Records dropped: {dropped_count}")
# print(f"Percentage of dataset removed: {percentage_dropped:.2f}%")

# combined_train_raw_df = filtered_df

In [5]:
# # drop unrealistic records of atm transfer out

# # current justification: it's not realisitc that someone can transfer money out of an ATM thousands of times in single day also it accounts for tiny 
# # TODO: find a source to backup the reasoning for removing this data

# initial_count = len(test_raw_df)

# # drop records with more than 10 atm transfer outs
# filtered_df = test_raw_df.drop(test_raw_df[test_raw_df['atm_transfer_out'] > 20].index)

# # Calculate dropped records
# dropped_count = initial_count - len(filtered_df)
# percentage_dropped = (dropped_count / initial_count) * 100

# print(f"Records dropped: {dropped_count}")
# print(f"Percentage of dataset removed: {percentage_dropped:.2f}%")

# test_raw_df = filtered_df

In [6]:
# # drop unrealistic records of atm transfer in

# # current justification: it's not realisitc that someone can transfer money out of an ATM thousands of times in single day also it accounts for tiny 
# # TODO: find a source to backup the reasoning for removing this data

# # drop records with more than 10 atm transfer ins
# filtered_df = combined_train_raw_df.drop(combined_train_raw_df[combined_train_raw_df['atm_transfer_in'] > 20].index)

# # Calculate dropped records
# dropped_count = initial_count - len(filtered_df)
# percentage_dropped = (dropped_count / initial_count) * 100

# print(f"Records dropped: {dropped_count}")
# print(f"Percentage of dataset removed: {percentage_dropped:.2f}%")

# combined_train_raw_df = filtered_df

In [7]:
# # drop unrealistic records of atm transfer in

# # current justification: it's not realisitc that someone can transfer money out of an ATM thousands of times in single day also it accounts for tiny 
# # TODO: find a source to backup the reasoning for removing this data

# # drop records with more than 10 atm transfer ins
# filtered_df = test_raw_df.drop(test_raw_df[test_raw_df['atm_transfer_in'] > 20].index)

# # Calculate dropped records
# dropped_count = initial_count - len(filtered_df)
# percentage_dropped = (dropped_count / initial_count) * 100

# print(f"Records dropped: {dropped_count}")
# print(f"Percentage of dataset removed: {percentage_dropped:.2f}%")

# test_raw_df = filtered_df

### Create the training dataset (customer_id level data)

Churn definition:
- No interactions with the bank for at 18 or more months since the max date in the dataset (2023-12-31)

In [8]:
# Ensure the 'date' column is in datetime format
combined_train_raw_df['date'] = pd.to_datetime(combined_train_raw_df['date'])

max_date_train = max(combined_train_raw_df['date'])
print(max_date_train)

2023-12-31 00:00:00


In [9]:
# Ensure the 'date' column is in datetime format
test_raw_df['date'] = pd.to_datetime(test_raw_df['date'])

max_date_test = max(test_raw_df['date'])
print(max_date_test)

2026-12-31 00:00:00


In [10]:
# Get the last interaction date for each customer
last_interaction = combined_train_raw_df.groupby('customer_id')['date'].max()

# Define the churn threshold
churn_date = pd.Timestamp(max_date_train) 
cutoff_date = churn_date - pd.DateOffset(months=18)

churn_status = last_interaction < cutoff_date

# Convert to DataFrame
df_train = churn_status.reset_index(name='churn_status')

churn_percentage = (df_train['churn_status'].sum() / len(df_train)) * 100
print(f"{churn_percentage:.2f}% of distinct customers since 2008-2023 have churned under our definition")

df_train.head()

19.94% of distinct customers since 2008-2023 have churned under our definition


,customer_id,churn_status
0,1,False
1,2,True
2,3,True
3,4,True
4,5,False


In [11]:
# Get the last interaction date for each customer
last_interaction = test_raw_df.groupby('customer_id')['date'].max()

# Define the churn threshold
churn_date = pd.Timestamp(max_date_test) # Training the model up to the 
cutoff_date = churn_date - pd.DateOffset(months=18)

# Determine churn status (True if last interaction was before cutoff_date)
churn_status = last_interaction < cutoff_date

# Convert to DataFrame
df_test = churn_status.reset_index(name='churn_status')

churn_percentage = (df_test['churn_status'].sum() / len(df_test)) * 100
print(f"{churn_percentage:.2f}% of distinct customers in 2024 have churned under our definition")

df_test.head()

14.96% of distinct customers in 2024 have churned under our definition


,customer_id,churn_status
0,1,False
1,2,False
2,3,True
3,4,True
4,5,True


## Feature Engineering

Features:
- customer age
- country - useless
- account age
- customer job category
- from competitor - useless

In [12]:
# This code compares our definition of churn to the instances of churn due to fraud (since we can assume that these are ACTUAL cases where a customer churned)

# Merge dataframes on customer_id
merged_df = combined_train_raw_df.merge(df_train, on="customer_id", how="inner")

# Categorizing based on churn_due_to_fraud and churn_status
conditions = [
    (merged_df["churn_due_to_fraud"] == True) & (merged_df["churn_status"] == True),
    (merged_df["churn_due_to_fraud"] == False) & (merged_df["churn_status"] == False),
    (merged_df["churn_due_to_fraud"] == True) & (merged_df["churn_status"] == False),
    (merged_df["churn_due_to_fraud"] == False) & (merged_df["churn_status"] == True)
]

categories = ["Both True", "Both False", "Fraud True, Churn False", "Fraud False, Churn True"]

merged_df["Category"] = np.select(conditions, categories, default="Unknown")
category_counts = merged_df["Category"].value_counts()

print(category_counts)

Category
Both False                 2655657
Fraud False, Churn True    1270374
Both True                       23
Fraud True, Churn False          4
Name: count, dtype: int64


In [13]:
# This code compares our definition of churn to the instances of churn due to fraud (since we can assume that these are ACTUAL cases where a customer churned)

# Merge dataframes on customer_id
merged_df = test_raw_df.merge(df_test, on="customer_id", how="inner")

# Categorizing based on churn_due_to_fraud and churn_status
conditions = [
    (merged_df["churn_due_to_fraud"] == True) & (merged_df["churn_status"] == True),
    (merged_df["churn_due_to_fraud"] == False) & (merged_df["churn_status"] == False),
    (merged_df["churn_due_to_fraud"] == True) & (merged_df["churn_status"] == False),
    (merged_df["churn_due_to_fraud"] == False) & (merged_df["churn_status"] == True)
]

categories = ["Both True", "Both False", "Fraud True, Churn False", "Fraud False, Churn True"]

merged_df["Category"] = np.select(conditions, categories, default="Unknown")
category_counts = merged_df["Category"].value_counts()

print(category_counts)

Category
Both False                 1284242
Fraud False, Churn True      76175
Fraud True, Churn False         48
Both True                        7
Name: count, dtype: int64


#### Customer Age

In [14]:
# # Compute age as of 2023-12-31
# combined_train_raw_df['date_of_birth'] = pd.to_datetime(combined_train_raw_df['date_of_birth'])
# age = (pd.Timestamp('2023-12-31') - combined_train_raw_df.groupby('customer_id')['date_of_birth'].min()).dt.days // 365

# # Convert to DataFrame and merge with df_train
# age_df = age.reset_index(name='age')
# df_train = df_train.merge(age_df, on='customer_id')
# len(df_train)

In [15]:
# # Compute age as of 2023-12-31
# test_raw_df['date_of_birth'] = pd.to_datetime(test_raw_df['date_of_birth'])
# age = (pd.Timestamp('2023-12-31') - test_raw_df.groupby('customer_id')['date_of_birth'].min()).dt.days // 365

# # Convert to DataFrame and merge with df_test
# age_df = age.reset_index(name='age')
# df_test = df_test.merge(age_df, on='customer_id')
# len(df_test)

#### Country

In [16]:
# # One-hot encode the 'country' column
# country_one_hot = pd.get_dummies(combined_train_raw_df['country'], prefix='country')

# # Merge one-hot encoded 'country' columns into df_train
# df_train = pd.merge(df_train, combined_train_raw_df[['customer_id']].drop_duplicates().merge(country_one_hot, left_index=True, right_index=True), on='customer_id')

In [17]:
# # One-hot encode the 'country' column
# country_one_hot = pd.get_dummies(test_raw_df['country'], prefix='country')

# # Merge one-hot encoded 'country' columns into df_test
# df_test = pd.merge(df_test, test_raw_df[['customer_id']].drop_duplicates().merge(country_one_hot, left_index=True, right_index=True), on='customer_id')

In [18]:
# ==================================================================
# TRANSACTION FEATURES
# ==================================================================
# 1. 30d Crypto Volume
combined_train_raw_df['daily_crypto_volume'] = combined_train_raw_df['crypto_in_volume'] + combined_train_raw_df['crypto_out_volume']

crypto_30d = (
    combined_train_raw_df
    .groupby('customer_id')
    .rolling('30D', on='date')
    ['daily_crypto_volume']
    .sum()
    .groupby('customer_id').last()
    .reset_index(name='30d_crypto_volume')
)

# 2. 90d ATM Withdrawal Trend (Slope)
def calculate_trend(series):
    if len(series) < 2: return np.nan
    x = np.arange(len(series))
    return np.polyfit(x, series, 1)[0]

atm_trend_90d = (
    combined_train_raw_df
    .groupby('customer_id')
    .rolling('90D', on='date')
    ['atm_transfer_out']
    .apply(calculate_trend, raw=True)
    .groupby('customer_id').last()
    .reset_index(name='90d_atm_trend')
)

# 3. Days Since Last Bank Transfer
last_transfer = (
    combined_train_raw_df[
        (combined_train_raw_df['bank_transfer_in'] > 0) |
        (combined_train_raw_df['bank_transfer_out'] > 0)
    ]
    .groupby('customer_id')['date'].max()
    .reset_index(name='last_transfer_date')
)
last_transfer['days_since_transfer'] = (
    pd.to_datetime(cutoff_date) - last_transfer['last_transfer_date']
).dt.days

In [19]:
# ==================================================================
# TRANSACTION FEATURES
# ==================================================================
# 1. 30d Crypto Volume
test_raw_df['daily_crypto_volume'] = test_raw_df['crypto_in_volume'] + test_raw_df['crypto_out_volume']

crypto_30d = (
    test_raw_df
    .groupby('customer_id')
    .rolling('30D', on='date')
    ['daily_crypto_volume']
    .sum()
    .groupby('customer_id').last()
    .reset_index(name='30d_crypto_volume')
)

# 2. 90d ATM Withdrawal Trend (Slope)
def calculate_trend(series):
    if len(series) < 2: return np.nan
    x = np.arange(len(series))
    return np.polyfit(x, series, 1)[0]

atm_trend_90d = (
    test_raw_df
    .groupby('customer_id')
    .rolling('90D', on='date')
    ['atm_transfer_out']
    .apply(calculate_trend, raw=True)
    .groupby('customer_id').last()
    .reset_index(name='90d_atm_trend')
)

# 3. Days Since Last Bank Transfer
last_transfer = (
    test_raw_df[
        (test_raw_df['bank_transfer_in'] > 0) |
        (test_raw_df['bank_transfer_out'] > 0)
    ]
    .groupby('customer_id')['date'].max()
    .reset_index(name='last_transfer_date')
)
last_transfer['days_since_transfer'] = (
    pd.to_datetime(cutoff_date) - last_transfer['last_transfer_date']
).dt.days

In [ ]:
# ==================================================================
# SUPPORT INTERACTION FEATURES
# ==================================================================
# 1. Extract Email CSAT Scores
combined_train_raw_df['email_csat'] = combined_train_raw_df['csat_scores'].apply(
    lambda x: x.get('email', np.nan) if isinstance(x, dict) else np.nan
)

# 2. Sort data chronologically
combined_train_raw_df = combined_train_raw_df.sort_values(['customer_id', 'date'])

# 3. Avg Email CSAT (60d)
email_csat_60d = (
    combined_train_raw_df
    .groupby('customer_id')
    .rolling('60D', on='date')['email_csat']
    .mean()
    .groupby('customer_id').last()
    .reset_index(name='avg_email_csat_60d')
)

# 4. Total Complaints (90d)
complaints_90d = (
    combined_train_raw_df
    .groupby('customer_id')
    .rolling('90D', on='date')['complaints']
    .sum()
    .groupby('customer_id').last()
    .reset_index(name='total_complaints_90d')
)

# 5. Phone Touchpoints (30d)
combined_train_raw_df['phone_touchpoints'] = combined_train_raw_df['touchpoints'].apply(
    lambda x: x.count('phone') if isinstance(x, list) else 0
)

phone_touch_30d = (
    combined_train_raw_df
    .groupby('customer_id')
    .rolling('30D', on='date')['phone_touchpoints']
    .sum()
    .groupby('customer_id').last()
    .reset_index(name='phone_touchpoints_30d')
)

# Fill NA values for customers with no support interactions
email_csat_60d['avg_email_csat_60d'] = email_csat_60d['avg_email_csat_60d'].fillna(
    combined_train_raw_df['email_csat'].median()
)
complaints_90d['total_complaints_90d'] = complaints_90d['total_complaints_90d'].fillna(0)
phone_touch_30d['phone_touchpoints_30d'] = phone_touch_30d['phone_touchpoints_30d'].fillna(0)

print("Email CSAT (60d):\n", email_csat_60d.head())
print("\nComplaints (90d):\n", complaints_90d.head())
print("\nPhone Touchpoints (30d):\n", phone_touch_30d.head())

Email CSAT (60d):
    customer_id  avg_email_csat_60d
0            1                 7.0
1            2                 7.0
2            3                 7.0
3            4                 7.0
4            5                 7.0

Complaints (90d):
    customer_id  total_complaints_90d
0            1                   0.0
1            2                   0.0
2            3                   0.0
3            4                   0.0
4            5                   0.0

Phone Touchpoints (30d):
    customer_id  phone_touchpoints_30d
0            1                    0.0
1            2                    0.0
2            3                    0.0
3            4                    0.0
4            5                    0.0


In [ ]:
# ==================================================================
# SUPPORT INTERACTION FEATURES
# ==================================================================
# 1. Extract Email CSAT Scores
test_raw_df['email_csat'] = test_raw_df['csat_scores'].apply(
    lambda x: x.get('email', np.nan) if isinstance(x, dict) else np.nan
)

# 2. Sort data chronologically
test_raw_df = test_raw_df.sort_values(['customer_id', 'date'])

# 3. Avg Email CSAT (60d)
email_csat_60d = (
    test_raw_df
    .groupby('customer_id')
    .rolling('60D', on='date')['email_csat']
    .mean()
    .groupby('customer_id').last()
    .reset_index(name='avg_email_csat_60d')
)

# 4. Total Complaints (90d)
complaints_90d = (
    test_raw_df
    .groupby('customer_id')
    .rolling('90D', on='date')['complaints']
    .sum()
    .groupby('customer_id').last()
    .reset_index(name='total_complaints_90d')
)

# 5. Phone Touchpoints (30d)
test_raw_df['phone_touchpoints'] = test_raw_df['touchpoints'].apply(
    lambda x: x.count('phone') if isinstance(x, list) else 0
)

phone_touch_30d = (
    test_raw_df
    .groupby('customer_id')
    .rolling('30D', on='date')['phone_touchpoints']
    .sum()
    .groupby('customer_id').last()
    .reset_index(name='phone_touchpoints_30d')
)

# Fill NA values for customers with no support interactions
email_csat_60d['avg_email_csat_60d'] = email_csat_60d['avg_email_csat_60d'].fillna(
    test_raw_df['email_csat'].median()
)
complaints_90d['total_complaints_90d'] = complaints_90d['total_complaints_90d'].fillna(0)
phone_touch_30d['phone_touchpoints_30d'] = phone_touch_30d['phone_touchpoints_30d'].fillna(0)

print("Email CSAT (60d):\n", email_csat_60d.head())
print("\nComplaints (90d):\n", complaints_90d.head())
print("\nPhone Touchpoints (30d):\n", phone_touch_30d.head())

Email CSAT (60d):
    customer_id  avg_email_csat_60d
0            1                 6.0
1            2                 6.0
2            3                 6.0
3            4                 6.0
4            5                 6.0

Complaints (90d):
    customer_id  total_complaints_90d
0            1                   0.0
1            2                   0.0
2            3                   0.0
3            4                   0.0
4            5                   0.0

Phone Touchpoints (30d):
    customer_id  phone_touchpoints_30d
0            1                    0.0
1            2                    0.0
2            3                    0.0
3            4                    0.0
4            5                    0.0


In [22]:
# ==================================================================
# FINANCIAL FEATURES 
# ==================================================================
# 1. Interest Rate Change (90d)
# Ensure data is sorted chronologically per customer
combined_train_raw_df = combined_train_raw_df.sort_values(['customer_id', 'date'])

interest_rates = (
    combined_train_raw_df
    .groupby('customer_id')
    .rolling('90D', on='date', closed='left')
    ['interest_rate']
    .mean()
    .groupby('customer_id').last()
    .reset_index(name='interest_rate_90d_ago')
)

current_interest = (
    combined_train_raw_df
    .groupby('customer_id')['interest_rate']
    .last()
    .reset_index(name='current_interest_rate')
)

interest_change = pd.merge(current_interest, interest_rates, on='customer_id')
interest_change['interest_rate_change_90d'] = (
    interest_change['current_interest_rate'] - interest_change['interest_rate_90d_ago']
)

# 2. Net Transfer Volume (30d)
combined_train_raw_df['net_transfer_volume'] = (
    combined_train_raw_df['bank_transfer_in_volume'] 
    - combined_train_raw_df['bank_transfer_out_volume']
)

net_transfer_30d = (
    combined_train_raw_df
    .groupby('customer_id')
    .rolling('30D', on='date')
    ['net_transfer_volume']
    .sum()
    .groupby('customer_id').last()
    .reset_index(name='net_transfer_30d')
)

In [23]:
# ==================================================================
# FINANCIAL FEATURES
# ==================================================================
# 1. Interest Rate Change (90d)
# Ensure data is sorted chronologically per customer
test_raw_df = test_raw_df.sort_values(['customer_id', 'date'])

interest_rates = (
    test_raw_df
    .groupby('customer_id')
    .rolling('90D', on='date', closed='left')
    ['interest_rate']
    .mean()
    .groupby('customer_id').last()
    .reset_index(name='interest_rate_90d_ago')
)

current_interest = (
    test_raw_df
    .groupby('customer_id')['interest_rate']
    .last()
    .reset_index(name='current_interest_rate')
)

interest_change = pd.merge(current_interest, interest_rates, on='customer_id')
interest_change['interest_rate_change_90d'] = (
    interest_change['current_interest_rate'] - interest_change['interest_rate_90d_ago']
)

# 2. Net Transfer Volume (30d)
test_raw_df['net_transfer_volume'] = (
    test_raw_df['bank_transfer_in_volume'] 
    - test_raw_df['bank_transfer_out_volume']
)

net_transfer_30d = (
    test_raw_df
    .groupby('customer_id')
    .rolling('30D', on='date')
    ['net_transfer_volume']
    .sum()
    .groupby('customer_id').last()
    .reset_index(name='net_transfer_30d')
)

In [24]:
# ==================================================================
# TEMPORAL FEATURES
# ==================================================================
# Month of cutoff date
df_train['month'] = cutoff_date.month

# Tenure Buckets (using latest tenure)
latest_tenure = combined_train_raw_df.groupby('customer_id')['tenure'].last().reset_index()
latest_tenure['tenure_bucket'] = pd.cut(latest_tenure['tenure'],
                                       bins=[0, 365, 1095, np.inf],
                                       labels=[0, 1, 2]) #'<1 year' = 0, '1-3 years' = 1, '>3 years' = 2

In [25]:
# ==================================================================
# TEMPORAL FEATURES
# ==================================================================
# Month of cutoff date
df_test['month'] = cutoff_date.month

# Tenure Buckets (using latest tenure)
latest_tenure = test_raw_df.groupby('customer_id')['tenure'].last().reset_index()
latest_tenure['tenure_bucket'] = pd.cut(latest_tenure['tenure'],
                                       bins=[0, 365, 1095, np.inf],
                                       labels=[0, 1, 2]) #'<1 year' = 0, '1-3 years' = 1, '>3 years' = 2

In [26]:
# ==================================================================
# DERIVED METRICS
# ==================================================================
# Engagement Score (example calculation)
engagement_components = pd.merge(
    crypto_30d[['customer_id', '30d_crypto_volume']],
    net_transfer_30d[['customer_id', 'net_transfer_30d']],
    on='customer_id'
)
engagement_components['engagement_score'] = (
    0.4 * engagement_components['30d_crypto_volume'] +
    0.6 * engagement_components['net_transfer_30d']
)

# Risk Flag (Large Withdrawal - top 5%)
withdrawal_threshold = combined_train_raw_df['bank_transfer_out_volume'].quantile(0.95)
risk_flag = combined_train_raw_df[combined_train_raw_df['bank_transfer_out_volume'] > withdrawal_threshold]\
            .groupby('customer_id').size()\
            .reset_index(name='large_withdrawals')
risk_flag['risk_flag'] = (risk_flag['large_withdrawals'] > 0).astype(int)

In [27]:
# ==================================================================
# DERIVED METRICS
# ==================================================================
# Engagement Score (example calculation)
engagement_components = pd.merge(
    crypto_30d[['customer_id', '30d_crypto_volume']],
    net_transfer_30d[['customer_id', 'net_transfer_30d']],
    on='customer_id'
)
engagement_components['engagement_score'] = (
    0.4 * engagement_components['30d_crypto_volume'] +
    0.6 * engagement_components['net_transfer_30d']
)

# Risk Flag (Large Withdrawal - top 5%)
withdrawal_threshold = test_raw_df['bank_transfer_out_volume'].quantile(0.95)
risk_flag = test_raw_df[test_raw_df['bank_transfer_out_volume'] > withdrawal_threshold]\
            .groupby('customer_id').size()\
            .reset_index(name='large_withdrawals')
risk_flag['risk_flag'] = (risk_flag['large_withdrawals'] > 0).astype(int)

In [28]:
# ==================================================================
# FINAL MERGE
# ==================================================================
feature_list = [
    crypto_30d, atm_trend_90d, last_transfer[['customer_id', 'days_since_transfer']],
    email_csat_60d, complaints_90d, phone_touch_30d,
    interest_change[['customer_id', 'interest_rate_change_90d']], 
    net_transfer_30d, latest_tenure[['customer_id', 'tenure_bucket']],
    engagement_components[['customer_id', 'engagement_score']], 
    risk_flag[['customer_id', 'risk_flag']]
]

df_train = df_train.merge(feature_list[0], on='customer_id', how='left')
for df in feature_list[1:]:
    df_train = df_train.merge(df, on='customer_id', how='left')

# Fill missing values
df_train.fillna({
    '30d_crypto_volume': 0,
    '90d_atm_trend': 0,
    'days_since_transfer': (cutoff_date - pd.to_datetime('2000-01-01')).days,
    'avg_email_csat_60d': df_train['avg_email_csat_60d'].median(),
    'risk_flag': 0
}, inplace=True)
len(df_train)

106179

In [29]:
# ==================================================================
# FINAL MERGE
# ==================================================================
feature_list = [
    crypto_30d, atm_trend_90d, last_transfer[['customer_id', 'days_since_transfer']],
    email_csat_60d, complaints_90d, phone_touch_30d,
    interest_change[['customer_id', 'interest_rate_change_90d']], 
    net_transfer_30d, latest_tenure[['customer_id', 'tenure_bucket']],
    engagement_components[['customer_id', 'engagement_score']], 
    risk_flag[['customer_id', 'risk_flag']]
]

df_test = df_test.merge(feature_list[0], on='customer_id', how='left')
for df in feature_list[1:]:
    df_test = df_test.merge(df, on='customer_id', how='left')

# Fill missing values
df_test.fillna({
    '30d_crypto_volume': 0,
    '90d_atm_trend': 0,
    'days_since_transfer': (cutoff_date - pd.to_datetime('2000-01-01')).days,  # Arbitrary old date
    'avg_email_csat_60d': df_test['avg_email_csat_60d'].median(),
    'risk_flag': 0
}, inplace=True)
len(df_test)

165155

### Unemployment Flag

In [30]:
# Get latest job status for each customer
latest_jobs = (
    combined_train_raw_df
    .sort_values(['customer_id', 'date'], ascending=[True, False])
    .groupby('customer_id')['job']
    .first()
    .reset_index()
)

# Create binary flag for unemployed status (latest record only)
latest_jobs['is_unemployed'] = (
    latest_jobs['job']
    .str.strip().str.lower()
    .eq('unemployed')
    .astype(int)
)

# Merge with training data
df_train = df_train.merge(
    latest_jobs[['customer_id', 'is_unemployed']],
    on='customer_id',
    how='left'
).fillna({'is_unemployed': 0})

print(df_train['is_unemployed'].value_counts())

is_unemployed
0    94780
1    11399
Name: count, dtype: int64


In [31]:
# Get latest job status for each customer
latest_jobs = (
    test_raw_df
    .sort_values(['customer_id', 'date'], ascending=[True, False])
    .groupby('customer_id')['job']
    .first()
    .reset_index()
)

# Create binary flag for unemployed status (latest record only)
latest_jobs['is_unemployed'] = (
    latest_jobs['job']
    .str.strip().str.lower()
    .eq('unemployed')
    .astype(int)
)

# Merge with testing data
df_test = df_test.merge(
    latest_jobs[['customer_id', 'is_unemployed']],
    on='customer_id',
    how='left'
).fillna({'is_unemployed': 0})

print(df_test['is_unemployed'].value_counts())

is_unemployed
0    135846
1     29309
Name: count, dtype: int64


### Export Train/Test sets to CSV

In [32]:
# Display final df_train
df_train.head()

,customer_id,churn_status,month,30d_crypto_volume,90d_atm_trend,days_since_transfer,avg_email_csat_60d,total_complaints_90d,phone_touchpoints_30d,interest_rate_change_90d,net_transfer_30d,tenure_bucket,engagement_score,risk_flag,is_unemployed
0,1,False,6,1602.602938,0.0,-242.0,6.0,0.0,0.0,NaN,734.663785,2,1081.839446,0.0,0
1,2,True,6,261.286543,0.0,-187.0,6.0,0.0,0.0,NaN,2.824867,1,106.209538,0.0,0
2,3,True,6,8490.917518,0.0,246.0,6.0,0.0,0.0,NaN,40025.197105,2,27411.485270,1.0,0
3,4,True,6,766.619416,0.0,128.0,6.0,0.0,0.0,NaN,365.017325,1,525.658162,0.0,0
4,5,False,6,304.032077,0.0,139.0,6.0,0.0,0.0,0.5,2698.584398,2,1740.763470,0.0,0


In [33]:
df_test.head()

,customer_id,churn_status,month,30d_crypto_volume,90d_atm_trend,days_since_transfer,avg_email_csat_60d,total_complaints_90d,phone_touchpoints_30d,interest_rate_change_90d,net_transfer_30d,tenure_bucket,engagement_score,risk_flag,is_unemployed
0,1,False,6,1602.602938,0.0,-242.0,6.0,0.0,0.0,NaN,734.663785,2,1081.839446,0.0,0
1,2,False,6,261.286543,0.0,-187.0,6.0,0.0,0.0,NaN,2.824867,1,106.209538,0.0,0
2,3,True,6,8490.917518,0.0,246.0,6.0,0.0,0.0,NaN,40025.197105,2,27411.485270,1.0,0
3,4,True,6,766.619416,0.0,128.0,6.0,0.0,0.0,NaN,365.017325,1,525.658162,0.0,0
4,5,True,6,304.032077,0.0,139.0,6.0,0.0,0.0,0.5,2698.584398,2,1740.763470,0.0,0


In [34]:
filename = f"../data/train_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
df_train.to_csv(filename, index=False)
print(f"Trainset saved as: {filename}")

Trainset saved as: ../data/train_2025-01-30_20-06-53.csv


In [35]:
filename = f"../data/test_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv"
df_test.to_csv(filename, index=False)
print(f"Testset saved as: {filename}")

Testset saved as: ../data/test_2025-01-30_20-06-54.csv
